In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from MF.IALSRecommender import IALSRecommender
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR
from MF.ALS import AlternatingLeastSquare


from Hybrid.hybridRec import HybridRecommender
#---------------

#----Model Load & Save----
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

In [2]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [3]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [4]:
featureList=list(d['feature_id'])

In [5]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [6]:
ICM

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [7]:
#Load
loaded_dataset = Dataset(None, None, None, None)
loaded_dataset.load_data("Salvataggi/")


urm_train = (loaded_dataset.get_urm_train())["urm_train"]
urm_validation = (loaded_dataset.get_urm_validation())["urm_validation"]
urm_test = loaded_dataset.get_urm_test()["urm_test"]

In [8]:
urm_train_validation = urm_train + urm_validation

**Hyper parameter TUNING**

In [9]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

**Tuning IALS**

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["num_factors"] = Integer(20, 400)
hyperparameters_range_dictionary["confidence_scaling"] = Categorical(["linear", "log"]) 
hyperparameters_range_dictionary["alpha"] = Real(0.001, 1)
hyperparameters_range_dictionary["epsilon"] = Real(0.01, 1)
hyperparameters_range_dictionary["reg"] = Real(1e-5, 1e-2)

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = IALSRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"

In [ ]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

**Evaluation Metrics**

In [10]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [11]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [12]:
#Params for the recommenders
cbf_param = {
    "knn": 380,
    "shrink": 9
}

user_cf_param = {
    "knn": 249,
    "shrink": 853
}

item_cf_param = {
    "knn": 119,
    "shrink": 430
}

slim_bpr_param = {   #work in progres....
    "learning_rate" : 0.00214168231523243,
    "epochs": 10,
    "nnz" : 0.39651777555716483,
    "knn": 1134
}

als_param = {   #work in progress...
    "n_factors": 242,
    "regularization": 2.0,
    "iterations": 15
}

Ials_param = {
    "num_factors" : 840,
    "confidence_scaling" : "linear",
    "alpha" : 0.40,
    "epsilon" : 0.28,
    "reg" : 0.0003
}


In [13]:
userCF = UserBasedCollaborativeFiltering(urm_train)

In [14]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

Similarity column 25975 ( 100 % ), 3038.34 column/sec, elapsed time 0.14 min


In [15]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.04790977256434664, 5007, 2940)

In [16]:
itemCF = ItemBasedCollaborativeFiltering(urm_train)

In [17]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

Similarity column 25975 ( 100 % ), 2851.42 column/sec, elapsed time 0.15 min


In [18]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.04681989284146422, 5007, 2940)

In [19]:
SlimBpr=SlimBPRRec(urm_train)

In [20]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

Get S SLIM BPR...
Epoch: 0


100%|██████████████████████████████████████████████████████████████████████████| 32424/32424 [00:18<00:00, 1787.37it/s]


Epoch: 1


100%|██████████████████████████████████████████████████████████████████████████| 32424/32424 [00:18<00:00, 1739.46it/s]


Epoch: 2


...

100%|██████████████████████████████████████████████████████████████████████████| 32424/32424 [00:18<00:00, 1724.31it/s]


Epoch: 19


100%|██████████████████████████████████████████████████████████████████████████| 32424/32424 [00:19<00:00, 1682.82it/s]


Keeping only knn = 1134 ...


100%|██████████████████████████████████████████████████████████████████████████| 25975/25975 [00:02<00:00, 9431.89it/s]


In [21]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.04609918915781799, 5007, 2940)

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [22]:
cbf = ContentBasedFiltering(urm_train,ICM)

In [23]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

Similarity column 25975 ( 100 % ), 1262.46 column/sec, elapsed time 0.34 min


In [24]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.028420201221214296, 5007, 2940)

In [25]:
ALS = AlternatingLeastSquare(urm_train)

In [26]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [27]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.05584213803203896, 5007, 2940)

In [ ]:
IALS = IALSRecommender(urm_train)

In [ ]:
IALS.fit(num_factors=Ials_param["num_factors"], confidence_scaling=Ials_param["confidence_scaling"], alpha=Ials_param["alpha"], epsilon=Ials_param["epsilon"], reg=Ials_param["reg"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(IALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Hybrid**

**Hybrid Recommender Tuning**

In [28]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [29]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["w_slim_bpr"] = Integer(0, 2000)

In [30]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = HybridRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [31]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train, ICM, userCF, itemCF, cbf, ALS, SlimBpr],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [user_cf_param, item_cf_param, cbf_param, slim_bpr_param, als_param, 37, 235, 98, 73],
    FIT_KEYWORD_ARGS = {}
)

In [32]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train, ICM, userCF, itemCF, cbf, ALS, SlimBpr],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [user_cf_param, item_cf_param, cbf_param, slim_bpr_param, als_param, 37, 235, 98, 73],
    FIT_KEYWORD_ARGS = {}
)

In [33]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 300
n_random_starts = int(n_cases*0.45)
metric_to_optimize = "MAP"

In [34]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 1689}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 29.45 sec. Users per second: 158
SearchBayesianSkopt: New best config found. Config 0: {'w_slim_bpr': 1689} - results: ROC_AUC: 0.1292475, PRECISION: 0.0250915, PRECISION_RECALL_MIN_DEN: 0.1076882, RECALL: 0.1058209, MAP: 0.0470767, MRR: 0.0891374, NDCG: 0.0710640, F1: 0.0405647, HIT_RATE: 0.2509154, ARHR: 0.0971470, NOVELTY: 0.0048391, AVERAGE_POPULARITY: 0.1089494, DIVERSITY_MEAN_INTER_LIST: 0.9846941, DIVERSITY_HERFINDAHL: 0.9984482, COVERAGE_ITEM: 0.3641193, COVERAGE_ITEM_CORRECT: 0.0264100, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1216811, DIVERSITY_GINI: 0.1216491, SHANNON_ENTROPY: 11.3865327, 

EvaluatorHoldout: Processed 5000 ( 99.86% ) in 30.82 sec. Users per second: 162
EvaluatorHoldout: Processed 5007 ( 100.00% ) in 30.88 sec. Users per second: 162
SearchBayesianSkopt: Best config evaluated with eval


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 29.5197
Function value obtained: -0.0487
Current minimum: -0.0556
Iteration No: 8 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 271}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 29.89 sec. Users per second: 155
SearchBayesianSkopt: Config 7 is suboptimal. Config: {'w_slim_bpr': 271} - results: ROC_AUC: 0.1457475, PRECISION: 0.0289899, PRECISION_RECALL_MIN_DEN: 0.1235130, RECALL: 0.1213570, MAP: 0.0546260, MRR: 0.1017781, NDCG: 0.0819801, F1: 0.0468001, HIT_RATE: 0.2898988, ARHR: 0.1120798, NOVELTY: 0.0048978, AVERAGE_POPULARITY: 0.0898318, DIVERSITY_MEAN_INTER_LIST: 0.9906034, DIVERSITY_HERFINDAHL: 0.9990390, COVERAGE_ITEM: 0.3671992, COVERAGE_ITEM_CORRECT: 0.0337247, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1374103, DIVERSITY_GINI: 0.1291202, SHANNON_ENTROPY: 11.6920757, 

Iteration No: 8 ended. Evaluation done at random point.
Time taken: 29.9325
Funct

Iteration No: 15 ended. Evaluation done at random point.
Time taken: 29.3032
Function value obtained: -0.0490
Current minimum: -0.0556
Iteration No: 16 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 1553}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 29.28 sec. Users per second: 159
SearchBayesianSkopt: Config 15 is suboptimal. Config: {'w_slim_bpr': 1553} - results: ROC_AUC: 0.1305171, PRECISION: 0.0254577, PRECISION_RECALL_MIN_DEN: 0.1092198, RECALL: 0.1073258, MAP: 0.0477533, MRR: 0.0903131, NDCG: 0.0720612, F1: 0.0411537, HIT_RATE: 0.2545768, ARHR: 0.0984997, NOVELTY: 0.0048398, AVERAGE_POPULARITY: 0.1084439, DIVERSITY_MEAN_INTER_LIST: 0.9848814, DIVERSITY_HERFINDAHL: 0.9984669, COVERAGE_ITEM: 0.3645428, COVERAGE_ITEM_CORRECT: 0.0270260, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1234428, DIVERSITY_GINI: 0.1219132, SHANNON_ENTROPY: 11.3949071, 

Iteration No: 16 ended. Evaluation done at random point.
Time taken: 29.3239


Iteration No: 23 ended. Evaluation done at random point.
Time taken: 29.5467
Function value obtained: -0.0493
Current minimum: -0.0558
Iteration No: 24 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 805}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 28.73 sec. Users per second: 162
SearchBayesianSkopt: Config 23 is suboptimal. Config: {'w_slim_bpr': 805} - results: ROC_AUC: 0.1387378, PRECISION: 0.0271376, PRECISION_RECALL_MIN_DEN: 0.1170607, RECALL: 0.1149875, MAP: 0.0507379, MRR: 0.0953999, NDCG: 0.0766730, F1: 0.0439118, HIT_RATE: 0.2713763, ARHR: 0.1046700, NOVELTY: 0.0048540, AVERAGE_POPULARITY: 0.1025935, DIVERSITY_MEAN_INTER_LIST: 0.9869270, DIVERSITY_HERFINDAHL: 0.9986714, COVERAGE_ITEM: 0.3672377, COVERAGE_ITEM_CORRECT: 0.0300674, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1303637, DIVERSITY_GINI: 0.1247697, SHANNON_ENTROPY: 11.4940405, 

Iteration No: 24 ended. Evaluation done at random point.
Time taken: 28.7819
Fu


Iteration No: 32 ended. Evaluation done at random point.
Time taken: 30.0545
Function value obtained: -0.0489
Current minimum: -0.0558
Iteration No: 33 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 1381}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 28.84 sec. Users per second: 161
SearchBayesianSkopt: Config 32 is suboptimal. Config: {'w_slim_bpr': 1381} - results: ROC_AUC: 0.1325166, PRECISION: 0.0256515, PRECISION_RECALL_MIN_DEN: 0.1104639, RECALL: 0.1085492, MAP: 0.0484909, MRR: 0.0914916, NDCG: 0.0730026, F1: 0.0414968, HIT_RATE: 0.2565152, ARHR: 0.0997998, NOVELTY: 0.0048405, AVERAGE_POPULARITY: 0.1077343, DIVERSITY_MEAN_INTER_LIST: 0.9851630, DIVERSITY_HERFINDAHL: 0.9984951, COVERAGE_ITEM: 0.3656208, COVERAGE_ITEM_CORRECT: 0.0274495, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1243236, DIVERSITY_GINI: 0.1225117, SHANNON_ENTROPY: 11.4100146, 

Iteration No: 33 ended. Evaluation done at random point.
Time taken: 28.8768

...

C:\Users\davel\anaconda3\envs\RecSysFramework\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


SearchBayesianSkopt: Testing config: {'w_slim_bpr': 1538}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 23.79 sec. Users per second: 195
SearchBayesianSkopt: Config 295 is suboptimal. Config: {'w_slim_bpr': 1538} - results: ROC_AUC: 0.1306263, PRECISION: 0.0255008, PRECISION_RECALL_MIN_DEN: 0.1093490, RECALL: 0.1074478, MAP: 0.0477979, MRR: 0.0903768, NDCG: 0.0721367, F1: 0.0412190, HIT_RATE: 0.2550075, ARHR: 0.0985635, NOVELTY: 0.0048401, AVERAGE_POPULARITY: 0.1084025, DIVERSITY_MEAN_INTER_LIST: 0.9848993, DIVERSITY_HERFINDAHL: 0.9984687, COVERAGE_ITEM: 0.3648508, COVERAGE_ITEM_CORRECT: 0.0271030, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1236945, DIVERSITY_GINI: 0.1220464, SHANNON_ENTROPY: 11.3967730, 

Iteration No: 296 ended. Search finished for the next optimal point.
Time taken: 27.8272
Function value obtained: -0.0478
Current minimum: -0.0558
Iteration No: 297 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 742}
Evalu

C:\Users\davel\anaconda3\envs\RecSysFramework\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


SearchBayesianSkopt: Testing config: {'w_slim_bpr': 0}
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 24.05 sec. Users per second: 193
SearchBayesianSkopt: Config 298 is suboptimal. Config: {'w_slim_bpr': 0} - results: ROC_AUC: 0.1427320, PRECISION: 0.0289253, PRECISION_RECALL_MIN_DEN: 0.1237887, RECALL: 0.1217464, MAP: 0.0548739, MRR: 0.1013160, NDCG: 0.0820821, F1: 0.0467446, HIT_RATE: 0.2892526, ARHR: 0.1110927, NOVELTY: 0.0049737, AVERAGE_POPULARITY: 0.0712909, DIVERSITY_MEAN_INTER_LIST: 0.9941693, DIVERSITY_HERFINDAHL: 0.9993955, COVERAGE_ITEM: 0.3674302, COVERAGE_ITEM_CORRECT: 0.0364196, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1374103, DIVERSITY_GINI: 0.1379547, SHANNON_ENTROPY: 11.9571592, 

Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 28.1071
Function value obtained: -0.0549
Current minimum: -0.0558
Iteration No: 300 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'w_slim_bpr': 1627}
EvaluatorH

TypeError: get_expected_ratings() missing 1 required positional argument: 'user_id'

In [ ]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

In [35]:
w = {
    "user_cf": 993,
    "item_cf": 513,
    "cbf": 44,
    "icm_svd": 0,
    "als": 0,
    "slim_bpr": 154,
    "elastic": 0
}

In [36]:
userCF = UserBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(urm_train_validation)

In [ ]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train_validation,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train_validation)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
recommender = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS, SlimBpr)

In [ ]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"], w_cbf=w["cbf"],w_als=w["als"], w_slim_bpr=w["slim_bpr"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [ ]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [ ]:
goodguys

In [ ]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

In [ ]:
def prepare_submission(users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend2(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [ ]:
submission = prepare_submission(users_to_recommend, urm_train_validation, recommender)

In [ ]:
submission

In [ ]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
write_submission(submission)